<a href="https://colab.research.google.com/github/snghynn/handsonml/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8_ch9_%EB%B9%84%EC%A7%80%EB%8F%84%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch9. 비지도학습
## 9.1 군집
- 군집: 비슷한 샘플을 구별해 하나의 클러스터 또는 비슷한 샘플의 그룹으로 할당하는 작업

- 분류와 달리 군집은 비지도 학습

### 9.9.1 k-평균
- k-평균: 반복 몇 번으로 레이블이 없는 데이터셋을 빠르고 효율적으로 클러스터로 묶을 수 있는 알고리즘

- 각 클러스터의 중심을 찾고 가장 가까운 클러스터에 샘플을 할당

In [19]:
from sklearn.datasets import make_blobs
import numpy as np


blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)

In [20]:
# 각 샘플은 다섯 개의 클러스터 중 하나에 할당
from sklearn.cluster import KMeans
k = 5 # 알고리즘이 찾을 클러스터 개수 지정
kmeans = KMeans(n_clusters = k)
y_pred = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


- 레이블: 알고리즘이 샘플에 할당한 클러스터의 인덱스

In [21]:
y_pred

array([4, 0, 1, ..., 3, 1, 0], dtype=int32)

In [22]:
y_pred is kmeans.labels_

True

In [23]:
# 이 알고리즘이 찾은 센트로이드 다섯 개
kmeans.cluster_centers_

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-1.46679593,  2.28585348],
       [-2.79290307,  2.79641063],
       [-2.80037642,  1.30082566]])

In [24]:
# 새로운 샘플에 가장 가까운 센트로이드의 클러스터 할당
X_new = np.array([[0,2],[3,2],[-3,3],[-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 3, 3], dtype=int32)

In [25]:
# transform(): 샘플과 각 센트로이드 사이의 거리 반환
kmeans.transform(X_new)

array([[2.81093633, 0.32995317, 1.49439034, 2.9042344 , 2.88633901],
       [5.80730058, 2.80290755, 4.4759332 , 5.84739223, 5.84236351],
       [1.21475352, 3.29399768, 1.69136631, 0.29040966, 1.71086031],
       [0.72581411, 3.21806371, 1.54808703, 0.36159148, 1.21567622]])

### k-평균 알고리즘

- 센트로이드가 주어진다 가정
- 데이터셋에 있는 모든 샘플에 가장 가까운 센트로이드의 클러스터 할당

- 레이블이나 센트로이드가 주어지지 않는 경우? : 처음에는 센트로이드를 랜덤하게 선정 -> 샘플에 레이블 할당, 업데이트 과정 반복
  - 센트로이드에 변화가 없을 때까지 반복


[알고리즘 작동 과정]
- 처음에 센트로이드를 랜덤하게 초기화
- 샘플에 레이블 할당
- 센트로이드 업데이트
- 샘플에 다시 레이블 할당

### 센트로이드 초기화 방법
- 센트로이드는 초기화가 중요

1. 센트로이드 위치 알 수 있다면 init 매개변수에 넘파이 배열 지정

In [26]:
import numpy as np
good_init = np.array([[-3,3],[-3,2],[-3,1],[-1,2],[0,2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)

2. 랜덤 초기화를 다르게 하여 알고리즘 여러번 실행 후 가장 좋은 솔루션 선택

- 랜덤 초기화 횟수 : n_init 매개변수로 조절(기본값 : 10)
- 성능 지표: 이너셔
 - 각 샘플과 가장 가까운 센트로이드 사이의 평균 제곱 거리
- KMeans 클래스는 알고리즘을 n_init번 실행하여 이너셔가 가장 낮은 모델 반환

### k-평균 속도 개선과 미니배치 k-평균

- 불필요한 거리 계산을 피하여 알고리즘 속도 향상: 삼각 부등식 사용

In [28]:
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


MiniBatchKMeans(n_clusters=5)

- 미니배치 k-평균 알고리즘이 일반 k-평균 알고리즘보다 훨씬 빠르지만 이너셔는 더 나쁨
  - 특히 클러스터의 개수가 증가할 때


### 최적의 클러스터 개수 찾기
- 이너셔 최소로 선택?
  - 이너셔는 k가 증가함에 따라 점점 작아지므로 k를 선택할 때 좋은 성능 지표가 아님
  - 클러스터가 늘어날수록 샘플은 가까운 센트로이드에 더 가깝게 됨

- 실루엣 점수: 모든 샘플에 대한 실루엣 계수의 평균

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

- 실루엣 다이어그램
  - 높이 : 클러스터가 포함하고 있는 샘플의 개수
  - 너비 : 클러스에 포함된 샘플의 정렬된 실루엣 계수(넓을수록 좋음)
  - 수직 파선 : 각 클러스터 개수에 해당하는 실루엣 점수

### 9.1.2 k-평균의 한계
- 장점: 속도가 빠르고 확장이 용이
- 단점
  - 최적이 아닌 솔루션을 피하려면 알고리즘을 여러 번 실행해야 함
  - 클러스터 개수를 지정해야 함
  - 크기나 밀집도가 서로 다르거나 원형이 아닐 경우 잘 작동하지 않음



### 9.1.3 군집을 사용한 이미지 분할
- 이미지 분할 : 이미지를 세그먼트 여러개로 분할하는 작업
- 시맨틱 분할 : 동일한 종류의 물체에 속한 모든 픽셀은 같은 세그먼트에 할당됨
- 색상 분할 : 동일한 색상을 가진 픽셀을 같은 세그먼트에 할당

In [33]:
import os
import urllib.request

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "unsupervised_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


images_path = os.path.join(PROJECT_ROOT_DIR, "images", "unsupervised_learning")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
filename = "ladybug.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/unsupervised_learning/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))
from matplotlib.image import imread
image = imread(os.path.join('images', 'unsupervised_learning', 'ladybug.png'))
image.shape

(533, 800, 3)

> - (높이, 너비, 컬러 채널 개수)
> - 컬러 채널 3개(RGB)

In [35]:
X = image.reshape(-1, 3)
kmeans = KMeans(n_clusters=8, random_state=42).fit(X)
segmented_img = kmeans.cluster_centers_[kmeans.labels_]
segmented_img = segmented_img.reshape(image.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


### 9.1.4 군집을 사용한 전처리
- 차원 축소에 효과적
- 지도 학습 알고리즘 적용 전 전처리 단계로 사용 가능


In [36]:
from sklearn.datasets import load_digits
X_digits, y_digits = load_digits(return_X_y = True)

In [37]:
# train, test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_digits, y_digits)

In [38]:
# 로지스틱 회귀 모델 훈련
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [39]:
# test set에서의 정확도 평가
log_reg.score(X_test, y_test)

0.9688888888888889

In [40]:
# k-평균을 전처리 단계로 사용해 더 좋아지는지 확인
from sklearn.pipeline import Pipeline

pipeline = Pipeline([ # 파이프라인 만들기
    # 훈련 세트를 50개의 클러스터로 모은 후 이미지를 50개 클러스터까지 거리로 바꿈
    ("kmeans",KMeans(n_clusters=50)),
    ("log_reg",LogisticRegression())
])
pipeline.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                ('log_reg', LogisticRegression())])

In [41]:
# 파이프라인 평가
pipeline.score(X_test, y_test)

0.9733333333333334

>- 오차율이 줄어듦

In [42]:
# 가장 좋은 k값: 교차 검증에서 가장 좋은 분류 성능을 내는 값
from sklearn.model_selection import GridSearchCV

param_grid = dict(kmeans__n_clusters=range(2,100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train,y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.5s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ...............................kmeans__n_clusters=2; total time=   0.5s


...

[CV] END ..............................kmeans__n_clusters=99; total time=   1.2s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[CV] END ..............................kmeans__n_clusters=99; total time=   1.2s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=50)),
                                       ('log_reg', LogisticRegression())]),
             param_grid={'kmeans__n_clusters': range(2, 100)}, verbose=2)

In [43]:
# 최선의 k값과 이때 파이프라인의 성능 확인
grid_clf.best_params_

{'kmeans__n_clusters': 92}

In [44]:
grid_clf.score(X_test, y_test)

0.9688888888888889

### 9.1.5 군집을 사용한 준지도 학습
- 준지도 학습: 레이블이 없는 데이터가 많고 레이블이 있는 데이터는 적을 때

In [46]:
# 숫자 데이터셋에서 레이블된 50개 샘플에 로지스틱 회귀 모델 훈련
n_labeled = 50
log_reg = LogisticRegression()
log_reg.fit(X_train[:n_labeled], y_train[:n_labeled])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [47]:
# test set에서 모델의 성능
log_reg.score(X_test, y_test)

0.8

>- 전체 데이터셋을 사용했을 때보다 낮은 정확도


- 개선
  - 훈련 세트를 50개의 클러스터로 모음
  - 각 클러스터에서 센트로이드에 가장 가까운 이미지 찾기(대표이미지)

In [48]:
k =50
kmeans = KMeans(n_clusters=k)
X_digits_dist = kmeans.fit_transform(X_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train[representative_digit_idx]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [49]:
y_train[representative_digit_idx]

array([6, 9, 7, 5, 3, 4, 1, 0, 9, 4, 2, 5, 1, 1, 8, 8, 7, 4, 5, 6, 5, 1,
       7, 0, 3, 3, 4, 2, 2, 7, 8, 1, 9, 6, 9, 8, 7, 8, 2, 4, 5, 6, 9, 4,
       3, 1, 8, 7, 4, 3])

In [50]:
# 이미지를 보고 수동으로 레이블 할당
y_representative_digits = np.array([6, 9, 7, 5, 3, 4, 1, 0, 9, 4, 2, 5, 1, 1, 8, 8, 7, 4, 5, 6, 5, 1,
       7, 0, 3, 3, 4, 2, 2, 7, 8, 1, 9, 6, 9, 8, 7, 8, 2, 4, 5, 6, 9, 4,
       3, 1, 8, 7, 4, 3])

>- 이 이미지들은 각 클러스터를 대표하는 이미지
- 성능이 높은지 확인

In [51]:
log_reg = LogisticRegression()
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9288888888888889

- 정확도가 올라감
- 샘플에 레이블을 부여하는 것은 비용이 많이 듦
- 무작위 샘플 대신 대표 샘플에 레이블 할당


- 레이블 전파 : 레이블을 동일한 클러스터에 있는 모든 샘플로 전파

In [52]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)
for i in range(k):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]

In [53]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9355555555555556

*************************************
----------------------------------

In [54]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), kmeans.labels_]
for i in range(k):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]

In [57]:
log_reg = LogisticRegression()
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9311111111111111

In [58]:
np.mean(y_train_partially_propagated == y_train[partially_propagated])

1.0

### 9.1.6 DBSCAN

In [59]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X, y=make_moons(n_samples=1000, noise=0.05)
dbscan=DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [60]:
dbscan.labels_

array([ 0,  2,  1,  2,  2,  7,  3,  3,  3, -1,  4,  3,  1,  1, -1,  3,  0,
        7, -1,  2,  2,  3, -1,  5,  5,  1,  2,  1,  6,  2, -1,  4,  5,  2,
        2,  1,  6,  3,  4,  4,  6,  7,  6, -1, -1,  7,  2,  5, -1,  1,  6,
        1,  6,  6,  0,  5,  5,  0,  2,  2,  5, -1,  6,  2,  3,  1,  3,  1,
        1, -1, -1,  1,  0,  5,  3,  4,  2,  2,  3,  6,  3,  0, -1,  6,  1,
        6,  2, -1,  3,  8,  7,  2,  1, -1,  6,  2,  6,  6,  1,  4,  7,  6,
        7,  6,  4,  4,  2,  6,  2,  2,  3,  1,  7, -1,  6,  0,  3,  1,  0,
        7,  7,  6,  6,  6,  2,  2,  1,  7, -1,  6,  2,  2, -1,  0,  1, -1,
        5,  6,  4,  3, -1,  3,  5,  5,  6, -1,  2,  7,  6,  6,  1,  7,  4,
        7,  5,  0,  4,  2,  6,  1,  4,  1,  0,  7,  7,  7,  6,  1,  1,  7,
        6,  6,  2,  1,  1,  6,  5,  4,  7,  6,  6,  7,  6,  6,  4,  1,  6,
        6,  6, -1,  7,  3,  3,  3,  2,  4,  1,  6,  6,  2, -1,  2,  2,  2,
        1,  6,  2,  6,  5, -1,  4,  1,  5,  6,  6,  0,  2,  4,  7,  0,  2,
        3,  7,  2,  7,  5

In [61]:
len(dbscan.core_sample_indices_)

798

In [62]:
# 핵심 샘플의 인덱스
dbscan.core_sample_indices_

array([  0,   2,   3,   4,   6,   7,  10,  11,  12,  15,  16,  19,  20,
        21,  24,  25,  26,  27,  28,  29,  31,  32,  33,  34,  35,  36,
        37,  38,  39,  40,  41,  42,  45,  46,  47,  49,  50,  51,  52,
        53,  54,  55,  57,  58,  60,  62,  63,  64,  66,  67,  71,  72,
        73,  74,  75,  76,  77,  78,  80,  81,  83,  84,  85,  86,  88,
        91,  92,  94,  95,  96,  97,  98,  99, 100, 102, 103, 104, 106,
       107, 108, 110, 111, 112, 114, 115, 116, 117, 118, 120, 121, 122,
       123, 124, 125, 126, 127, 129, 130, 131, 134, 136, 137, 138, 139,
       141, 142, 143, 144, 146, 147, 148, 149, 150, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 163, 164, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       201, 203, 204, 205, 206, 207, 208, 210, 211, 212, 214, 215, 216,
       218, 220, 221, 222, 223, 224, 226, 228, 229, 230, 231, 23

In [63]:
dbscan.components_

array([[-0.39978336,  0.98034499],
       [ 0.32713278, -0.19753423],
       [ 1.32813858, -0.43676659],
       ...,
       [-0.6557953 ,  0.69934953],
       [ 1.52659459, -0.41291296],
       [ 0.47764787, -0.34365721]])

In [64]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

KNeighborsClassifier(n_neighbors=50)

In [65]:
X_new=np.array([[-0.5, 0], [0, 0.5], [1, -0.1], [2,1]])
knn.predict(X_new)

array([3, 1, 6, 7])

In [66]:
y_dist, y_pred_idx=knn.kneighbors(X_new, n_neighbors=1)
y_pred=dbscan.labels_[dbscan.core_sample_indices_][y_pred_idx]
y_pred[y_dist>0.2]=-1
y_pred.ravel()

array([-1,  1,  6, -1])

## 9.2 가우시안 혼합

In [67]:
from sklearn.mixture import GaussianMixture

gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)

GaussianMixture(n_components=3, n_init=10)

In [68]:
gm.weights_

array([0.18885092, 0.19511964, 0.61602944])

In [69]:
gm.means_

array([[-0.78174481,  0.52975236],
       [ 1.76108172, -0.04089931],
       [ 0.49198824,  0.25626039]])

In [70]:
gm.covariances_

array([[[ 0.04002064,  0.05128004],
        [ 0.05128004,  0.08054304]],

       [[ 0.04432668,  0.05600776],
        [ 0.05600776,  0.0846153 ]],

       [[ 0.18639641, -0.12044314],
        [-0.12044314,  0.29374264]]])

In [71]:
gm.converged_

True

In [72]:
gm.n_iter_

18

In [73]:
gm.predict(X)

array([0, 2, 2, 2, 1, 1, 0, 0, 0, 2, 0, 0, 2, 2, 1, 0, 2, 1, 2, 2, 1, 0,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 1, 2, 2,
       2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 0, 2,
       0, 2, 2, 1, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2,
       0, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 0, 1, 2, 1, 2, 0, 0, 2, 2, 1, 2,
       0, 2, 1, 2, 2, 2, 0, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 0, 1,
       2, 2, 0, 2, 2, 2, 0, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 0, 0, 2, 0, 2, 2,
       2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 1, 0,
       1, 0, 1, 2, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2, 0, 2, 0,
       2, 2, 2, 1, 0, 0, 2, 0, 1, 2, 1, 2, 2, 2, 2, 0, 2, 1, 2, 0, 2, 2,
       2, 2, 2, 0, 2, 2, 0, 2, 1, 0, 1, 0, 2, 1, 1, 2, 0, 2, 2, 2, 2, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2,

In [74]:
gm.predict_proba(X)

array([[7.74889760e-001, 1.40176516e-243, 2.25110240e-001],
       [1.14472354e-190, 3.64902589e-004, 9.99635097e-001],
       [3.44556625e-074, 5.29927125e-053, 1.00000000e+000],
       ...,
       [1.61699301e-045, 2.35437082e-083, 1.00000000e+000],
       [3.48929951e-262, 9.99608309e-001, 3.91690685e-004],
       [1.32848365e-245, 9.99655920e-001, 3.44079972e-004]])

In [75]:
X_new, y_new = gm.sample(6)
X_new

array([[-0.4272623 ,  0.91046432],
       [ 1.50533594, -0.42186202],
       [ 1.76262778,  0.18506388],
       [ 0.15665068,  0.41841161],
       [ 0.50674325,  0.15909928],
       [ 0.80832896,  0.56289437]])

In [76]:
y_new

array([0, 1, 1, 2, 2, 2])

In [77]:
# 확률 밀도 함수의 로그를 예측
gm.score_samples(X)

array([-1.40866229, -2.00300078, -1.51590166, -1.98572983, -0.42922651,
       -0.74203214, -1.24485026, -0.80767539, -1.2335882 , -1.60344879,
       -0.56197697, -0.51407747, -1.68403356, -1.81653773, -3.69642236,
       -1.64960599, -2.33454223, -0.6679221 , -1.56043195, -1.4974285 ,
       -0.49422972, -1.43809989, -1.84783108, -1.85777239, -1.99373067,
       -1.30020655, -1.51957184, -1.58702847, -1.79733011, -1.59749333,
       -1.55226028,  0.12380208, -1.641747  , -1.72158743, -2.25087003,
       -1.64436293, -1.42758684, -2.03406796, -0.193155  , -0.23243771,
       -1.54691978,  0.02625395, -1.82705826, -1.21055723, -2.02027751,
       -0.39758863, -1.99953822, -1.57353605, -1.89248651, -1.41606471,
       -1.58305811, -1.7553026 , -1.78609644, -1.72854545, -2.22882774,
       -1.87330728, -1.40232844, -1.2700299 , -1.97792871, -1.69748635,
       -2.1128079 , -1.71034324, -1.43521255, -0.18163184, -0.40508298,
       -1.41362958, -0.4989695 , -1.38686528, -1.90672504, -2.28

### 9.2.1 가우시안 혼합을 사용한 이상치 탐지

In [78]:
densities=gm.score_samples(X)
density_threshold=np.percentile(densities, 4)
anomalies=X[densities < density_threshold]

### 9.2.2 클러스터 개수 선택하기

In [79]:
gm.bic(X)

2718.2020168476924

In [80]:
gm.aic(X)

2634.770177104996

### 9.2.3 베이즈 가우시안 혼합 모델

In [81]:
from sklearn.mixture import BayesianGaussianMixture
bgm=BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(X)
np.round(bgm.weights_, 2)

array([0.09, 0.  , 0.13, 0.18, 0.11, 0.1 , 0.12, 0.14, 0.12, 0.  ])